# Notebook inicial para o projeto final de Aprendizagem de Máquina.

1. Escolher o tema e os dados.
2. Definir bem o workflow/pipeline de dados, como vamos realizar o nosso trabalho.
3. Botar a mão na massa.

## Endpoints que iremos utilizar:

* Link com todos os endpoints: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx

* ListarProposicoes: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoes

* ListarProposicoesVotadasEmPlenario: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoesVotadasEmPlenario

* ObterVotacaoProposicaoPorID: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicaoPorID

* ObterVotacaoProposicao: https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterProposicaoPorID

## Pipeline de uso dos endpoints:

1. Pegar as proposições e colocar num csv. (Definir os anos. Vamos focar em Projetos de Lei.)
2. Criar nova(s) coluna(s) em relação ao fato de aquela proposição ter sido votada
3. Decidir se filtramos só as que foram votadas ou se vamos utilizar o fato de ser votada ou não como uma feature.
4. Ver se há necessidade de criar novas colunas.
5. Adicionar os resultados das votações.
6. Pensar nos próximos passos.


In [4]:
import requests, time, os
from datetime import datetime
import xml.etree.ElementTree as ET
import xml_to_df
import pandas as pd

In [5]:
df = pd.read_csv('data/ListaProposicoes_2013_2024.csv')

#df = pd.read_csv('data/ListaProposicoesVotadasPlenario_2013_2024.csv')

In [6]:
df.columns

Index(['id', 'nome', 'tipoProposicao_id', 'tipoProposicao_sigla',
       'tipoProposicao_nome', 'numero', 'ano', 'orgaoNumerador_id',
       'orgaoNumerador_sigla', 'orgaoNumerador_nome', 'datApresentacao',
       'txtEmenta', 'txtExplicacaoEmenta', 'regime_codRegime',
       'regime_txtRegime', 'apreciacao_id', 'apreciacao_txtApreciacao',
       'autor1_txtNomeAutor', 'autor1_idecadastro', 'autor1_codPartido',
       'autor1_txtSiglaPartido', 'autor1_txtSiglaUF', 'qtdAutores',
       'ultimoDespacho_datDespacho', 'ultimoDespacho_txtDespacho',
       'situacao_id', 'situacao_descricao', 'situacao_orgao',
       'situacao_principal', 'indGenero', 'qtdOrgaosComEstado'],
      dtype='object')

In [7]:
import concurrent.futures
import requests

url = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterProposicaoPorID'
responses = {}
dfs = {}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def fetch_data(id_prop, url, headers):
    params = {'idProp': id_prop}
    response = xml_to_df.get_data(url, params, headers)
    
    if response.status_code != 200:
        print(f'Erro ao extrair dados de id {id_prop}')
        return id_prop, None
    
    df_id = xml_to_df.data_to_df(response, ObterProposicaoPorID='porId')
    return id_prop, (response, df_id)

for ano in range(2021, 2025):
    df_ano = df[df['ano'] == ano]
    unique_ids = df_ano['id'].unique()

    # Parallel processing for fetching data
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_data, id_prop, url, headers): id_prop for id_prop in unique_ids}

        for future in concurrent.futures.as_completed(futures):
            id_prop = futures[future]
            try:
                result = future.result()
                if result and result[1]:
                    responses[result[0]], dfs[result[0]] = result[1]
                    print(f'Dados de id {result[0]} extraídos com sucesso')
            except Exception as e:
                print(f'Erro ao processar dados de id {id_prop}: {e}')
    
    # Merging DataFrames
    if dfs:
        merged_df_ano = pd.concat(dfs.values(), ignore_index=True)
        xml_to_df.save_csv(merged_df_ano, f"ObterProposicoes_{ano}.csv")
        print(f'Arquivo ObterProposicoes_{ano}.csv salvo com sucesso')
    else:
        print(f'Nenhum dado disponível para o ano {ano}')


Dados de id 2313527 extraídos com sucesso
Dados de id 2313515 extraídos com sucesso
Dados de id 2313512 extraídos com sucesso
Dados de id 2313496 extraídos com sucesso
Dados de id 2313522 extraídos com sucesso
Dados de id 2313524 extraídos com sucesso
Dados de id 2313525 extraídos com sucesso
Dados de id 2313514 extraídos com sucesso
Dados de id 2313526 extraídos com sucesso
Dados de id 2313523 extraídos com sucesso
Dados de id 2313494 extraídos com sucesso
Dados de id 2313489 extraídos com sucesso
Dados de id 2313495 extraídos com sucesso
Dados de id 2313453 extraídos com sucesso
Dados de id 2313425 extraídos com sucesso
Dados de id 2313450 extraídos com sucesso
Dados de id 2313449 extraídos com sucesso
Dados de id 2313413 extraídos com sucesso
Dados de id 2313409 extraídos com sucesso
Dados de id 2313476 extraídos com sucesso
Dados de id 2313477 extraídos com sucesso
Dados de id 2313395 extraídos com sucesso
Dados de id 2313403 extraídos com sucesso
Dados de id 2313394 extraídos com 

In [11]:
OP_PATH = 'data/obter_proposicoes/'

merged_df = pd.DataFrame()

for file in os.listdir(OP_PATH):
    if file.endswith('.csv'):
        df = pd.read_csv(OP_PATH + file)
        print(f'{file}: {df.shape[0]} linhas')
        merged_df = pd.concat([merged_df, df], ignore_index=True)

ObterProposicoes_2017.csv: 12832 linhas
ObterProposicoes_2014.csv: 3399 linhas
ObterProposicoes_2020.csv: 24564 linhas
ObterProposicoes_2015.csv: 7643 linhas
ObterProposicoes_2021.csv: 3907 linhas
ObterProposicoes_2022.csv: 6361 linhas
ObterProposicoes_2024.csv: 15137 linhas
ObterProposicoes_2018.csv: 14642 linhas
ObterProposicoes_2016.csv: 10167 linhas
ObterProposicoes_2019.csv: 19913 linhas
ObterProposicoes_2023.csv: 11732 linhas
ObterProposicoes_2013.csv: 2102 linhas


In [15]:
merged_df.to_csv('data/ObterProposicoes_2013_2024.csv', index=False)